In [14]:
import os 
import polars as pl 
import sys 
import json 
import numpy as np
sys.path.append("..")

from settings import (
    REGRESSION_TARGET,CLASSIFICATION_TARGET, random_state, PROJECT_PATH
)

from sklearn.model_selection import cross_validate, TimeSeriesSplit
from xgboost import XGBClassifier

In [15]:
transactions = pl.read_parquet(
    os.path.join(PROJECT_PATH, "transactions_post_feature_engineering.parquet")
)

with open("../features_used.json", "r") as f:
    feature_names = json.load(f)

# %%
selected_region = "nom_region_Occitanie"
region_transactions = transactions.filter(pl.col(selected_region) == 1)

In [16]:

X = region_transactions.drop([REGRESSION_TARGET, CLASSIFICATION_TARGET])
y_classification = region_transactions[CLASSIFICATION_TARGET]


In [17]:
def perform_cross_validation(
    X: pl.DataFrame,
    y: pl.Series,
    model,
    cross_val_type,
    scoring_metrics: tuple,
    return_estimator=False,
    groups=None,
):
    scores = cross_validate(
        model,
        X.to_numpy(),
        y.to_numpy(),
        cv=cross_val_type,
        return_train_score=True,
        return_estimator=return_estimator,
        scoring=scoring_metrics,
        groups=groups,
    )

    for metric in scoring_metrics:
        print(
            "Average Train {metric} : {metric_value}".format(
                metric=metric,
                metric_value=np.mean(scores["train_" + metric]),
            )
        )
        print(
            "Train {metric} Standard Deviation : {metric_value}".format(
                metric=metric, metric_value=np.std(scores["train_" + metric])
            )
        )

        print(
            "Average Test {metric} : {metric_value}".format(
                metric=metric, metric_value=np.mean(scores["test_" + metric])
            )
        )
        print(
            "Test {metric} Standard Deviation : {metric_value}".format(
                metric=metric, metric_value=np.std(scores["test_" + metric])
            )
        )

    return scores

In [18]:
# Ce serait une erreur de travailler avec une donnée non-triée quand on veut utiliser cette variante
X = X.sort(["annee_transaction", "mois_transaction"])


In [19]:
xgboost_classifier = XGBClassifier(random_state=random_state)
classification_scoring_metrics = ("precision", "recall", "roc_auc")

TimeSeriesSplit() est une méthode de validation croisée utilisée pour évaluer la performance d'un modèle sur des données avec une dimension temporelle. Elle divise les données en segments temporels non chevauchant, en utilisant les données du passé pour prédire les données du futur.

Lorsque vous utilisez TimeSeriesSplit(), vous spécifiez le nombre de splits (ou folds) que vous souhaitez créer. Par exemple, si vous avez 10 splits, votre jeu de données sera divisé en 10 segments temporels.

Le premier split utilise les données du début du jeu de données pour l'entraînement et les données suivantes pour le test. Le deuxième split utilise les données du début jusqu'à la fin du premier split pour l'entraînement et les données suivantes pour le test. Cette méthode est répétée jusqu'à ce que tous les splits soient créés.

Cela permet d'évaluer la performance du modèle sur des données futures, en s'assurant qu'il n'a pas accès à des informations sur le futur lors de la phase d'entraînement. Cela est particulièrement important pour les données temporelles, car les modèles doivent être capables de prédire l'avenir en se basant sur les données du passé.

In [20]:
scores_xgboost = perform_cross_validation(
    X=X[[col for col in feature_names if col in X.columns]],
    y=y_classification,
    model=xgboost_classifier,
    cross_val_type=TimeSeriesSplit(), #Par défaut, le nombre de folds est 5
    scoring_metrics=classification_scoring_metrics,
)

Average Train precision : 0.805238579824026
Train precision Standard Deviation : 0.025844640476000855
Average Test precision : 0.4080283232385069
Test precision Standard Deviation : 0.020898104786274272
Average Train recall : 0.5052595507758886
Train recall Standard Deviation : 0.11411039229388611
Average Test recall : 0.2565957137562226
Test recall Standard Deviation : 0.05806081128105084
Average Train roc_auc : 0.8299132079107723
Train roc_auc Standard Deviation : 0.0448855972797487
Average Test roc_auc : 0.4942290932104174
Test roc_auc Standard Deviation : 0.007638449053208446


A premier abord, nous avons une dégradation considérable de la performance du modèle en test, ce qui suggère un fort overfit et une incapacité de prédire le contexte immobilier changeant avec le temps. Cela pourrait signifier que notre modèle nécéssiterait un réentrainement régulier, ou un feature engineering plus riche pour prendre en considération des informations invisibles au modèle aujourd'hui. 

Si nous souhaitons pousser l'analyse plus loin, il faudrait regarder les dates exactes de début et de fin de chaque fold et regarder à quel point le contexte de la donnée à changé ! Nous savons que notre jeu de données couvre la période Juin 2018-Decembre 2022. Entre le covid et l'inflation qui a suivi, on est sur que les folds vont avoir des contextes immobiliers très différents. 

Malheureusement, la fonction cross_validate ne permets pas cela ! Il faudrait utiliser une fonction avec une boucle for comme celle ci-dessous. Cela permetrait de récupérer les indices des lignes et d'analyser les périodes temporelles ainsi que leurs features en détail 

In [ ]:
tscv = TimeSeriesSplit()
for i, (train_index, test_index) in enumerate(tscv.split(X)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")